In [ ]:
import pandas as pd
from tqdm import tqdm
import ast

In [ ]:
com = pd.read_csv('main_comments.csv')
main_df = pd.read_pickle('../data/reddit_corpus_balanced_filtered.gzip', compression='gzip')

In [ ]:
df = pd.DataFrame(columns=['user_id', 'doc_id', 'text', 'fn', 'annotations', 'comments'])
doc_ids = set(com['doc_id'])
for r in tqdm(main_df.iterrows(), total=len(main_df)):
    d = r[1]
    for doc in d['documents']:
        if len(doc[4]) > 0:
            temp = {'user_id': d['user_id'], 'doc_id': doc[0], 'text': doc[1],
                    'annotations':doc[4], 'comments': {},
                   'fn': int(max(i[1] for i in doc[4]))}
            
            c = com[com['doc_id'] == doc[0]]
            
            # Find post from factroid dataset in comments, skip if not in comments
            if len(c) == 1:
                c = c.iloc[0, :]
            else:
                continue
            
            if c['comments'] == 'Empty':
                pass
            else:
                temp['comments'] = ast.literal_eval(c['comments'])
                    
            df = df.append(temp, ignore_index=True)

df

In [ ]:
df.to_pickle('factroid_with_comments.pkl')

In [ ]:
# Save csv from which to calc nvidia emotions

main = df.iloc[:,:3]
main.to_csv('main_posts_78387.csv')

In [ ]:
comments = []
for com in tqdm(df['comments']):
    for c in com:
        comments.append({'id': c, 'text': com[c]})
        
comments = pd.DataFrame(data=comments, columns=['id', 'text'])
comments.to_csv('comments_on_posts_101390')